In [51]:
import io_tools as iot
import os
import numpy as np
from time import time
import sys
import copy
import pdb
import cPickle as pickle
import gzip

from transformers import *
from sklearn.cross_validation import StratifiedKFold
import xgboost as xgb

DATA_DIR = '/media/raid_arr/data/otto/data/'
DB_OUT_TRAIN = os.path.join(DATA_DIR, 'train_lvl')
DB_OUT_TEST = os.path.join(DATA_DIR, 'test_lvl')
DB_OUT_ALL = os.path.join(DATA_DIR, 'all_lvl')
DB_OUT_ALL0 = os.path.join(DATA_DIR, 'all0_lvl')

def _load_data():
#     data = pickle.load(open(os.path.join(DATA_DIR, 'transformed_data.p'), 'rb'))
#     data = pickle.load(gzip.open(os.path.join(DATA_DIR, 'log_data.pgz'), 'rb'))
    data = pickle.load(gzip.open(os.path.join(DATA_DIR, 'iden-log-anscombe_data.pgz'), 'rb'))
    #data = pickle.load(gzip.open(os.path.join(DATA_DIR, 'iden-log-anscombe-inv_data.pgz'), 'rb'))
    return data



In [47]:
try:
    ids_train, feats_train, labels_train
except NameError:
    ids_train = np.load(os.path.join(iot.DATA_DIR, 'train_ids.npy'))
    feats_train = np.load(os.path.join(iot.DATA_DIR, 'train_feats.npy')).astype(float)
    labels_train = np.load(os.path.join(iot.DATA_DIR, 'train_labels_enc.npy'))

try:
    ids_test, feats_test
except NameError:
    ids_test = np.load(os.path.join(iot.DATA_DIR, 'test_ids.npy'))
    feats_test = np.load(os.path.join(iot.DATA_DIR, 'test_feats.npy')).astype(float)   
    
skf = StratifiedKFold(labels_train, n_folds=5, shuffle=True)
# All
feats_all = np.r_[feats_train, feats_test]
labels_all = np.r_[labels_train, -1*np.ones(len(ids_test))].astype(int)

# All minus test0
train_ind, val_ind = iter(skf).next()
feats_all0 = np.r_[feats_train[train_ind, :], feats_test]
labels_all0 = np.r_[labels_train[train_ind], -1*np.ones(len(ids_test))].astype(int)


feats_fold_train = feats_train[train_ind, :]
labels_fold_train = labels_train[train_ind]
feats_fold_val = feats_train[val_ind, :]
labels_fold_val = labels_train[val_ind]


In [103]:
# pipe = make_pipeline(IdentityTformer())
pipe = make_pipeline(AnscombeTformer())
pipe.fit(np.r_[feats_train[train_ind, :], feats_test])

X_train = pipe.transform(feats_fold_train)
y_train = labels_fold_train

X_valid = pipe.transform(feats_fold_val)
y_valid = labels_fold_val

X_test = pipe.transform(feats_test)
y_test = -1*np.ones(len(feats_test), dtype=int)

data = ((X_train, y_train),
        (X_valid, y_valid),
        (X_test, y_test))

In [104]:
# Load data
# data = _load_data()
X_train, y_train = data[0]
X_valid, y_valid = data[1]
X_test, y_test = data[2]

# Prep data for xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_valid, label=y_valid)
dtest = xgb.DMatrix(X_test, label=y_test)

# # Save binary for faser loading next time?
# dtrain.save_binary(os.path.join(DATA_DIR, 'train.buffer'))
# dvalid.save_binary(os.path.join(DATA_DIR, 'valid.buffer'))
# dtest.save_binary(os.path.join(DATA_DIR, 'test.buffer'))

In [137]:
# Parameters for XGB
params = {
    'num_class': len(np.unique(y_train)),
    'silent': 0,    # 0 is verbose
    'nthread': 7,    # max if not set
    
    'bst:eta': 0.2,
    'bst:gamma': 1,
    'bst:max_depth': 11,
    'bst:min_child_weight': 3,
    'bst:max_delta_step': 0,
    'bst:subsample': 0.9,    # Row (observation) subsample
    'bst:colsample_bytree': 0.6,    # Col (feature) subsample
    
    'objective':'multi:softprob',
    'eval_metric': 'mlogloss',
    
    'seed': 0,
}
plst = params.items()
evallist  = [(dtrain,'train'), (dvalid,'valid')]

In [138]:
tic = time()
num_round = 500
bst = xgb.train(plst, dtrain, num_round, evallist,
               early_stopping_rounds=20)
toc = time() - tic
print 'Train time: %g s' % toc

Will train until valid error hasn't decreased in 20 rounds.
[0]	train-mlogloss:1.721699	valid-mlogloss:1.737676
[1]	train-mlogloss:1.472023	valid-mlogloss:1.503418
[2]	train-mlogloss:1.288700	valid-mlogloss:1.332945
[3]	train-mlogloss:1.146573	valid-mlogloss:1.199928
[4]	train-mlogloss:1.033793	valid-mlogloss:1.095962
[5]	train-mlogloss:0.940083	valid-mlogloss:1.011610
[6]	train-mlogloss:0.866177	valid-mlogloss:0.943941
[7]	train-mlogloss:0.801193	valid-mlogloss:0.885430
[8]	train-mlogloss:0.746961	valid-mlogloss:0.838235
[9]	train-mlogloss:0.700766	valid-mlogloss:0.798246
[10]	train-mlogloss:0.661434	valid-mlogloss:0.764214
[11]	train-mlogloss:0.627885	valid-mlogloss:0.734685
[12]	train-mlogloss:0.596703	valid-mlogloss:0.708416
[13]	train-mlogloss:0.569816	valid-mlogloss:0.685972
[14]	train-mlogloss:0.545626	valid-mlogloss:0.665607
[15]	train-mlogloss:0.525892	valid-mlogloss:0.648832
[16]	train-mlogloss:0.505383	valid-mlogloss:0.632314
[17]	train-mlogloss:0.488333	valid-mlogloss:0.619

Train time: 174.732 s


Stopping. Best iteration:
[204]	train-mlogloss:0.177534	valid-mlogloss:0.454933



In [88]:
X_train

array([[  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  1.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       ..., 
       [  4.,   0.,   0., ...,   0.,   1.,   0.],
       [  1.,   0.,   0., ...,   3.,  10.,   0.],
       [  0.,   0.,   0., ...,   0.,   2.,   0.]])